In [1]:
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
kpis = pd.read_csv("simulation_data/gateway_viability_sweep_ag1_.csv", index_col=0)
from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from gvs_threshold_calc import calc_gvs_thresh_ineq_met

/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fitness_func = calc_gvs_thresh_ineq_met


In [3]:
params_to_evolve = {
                     "session_token_bucket_coefficient" : [25, 400],
                     "gateway_fee_per_relay" : [20, 30], 
                     "application_fee_per_relay" : [20, 30],
                      "gateway_minimum_stake" :  [100000 * 1e6, 200000 * 1e6],
                      "application_minimum_stake" : [100000 * 1e6,
                                                     200000 * 1e6,
                                                            ]
                   }

params_for_setup_only = {
                      "application_max_number" : [5, 20, 100],
                   "relays_per_session_gamma_distribution_scale" : [100000,
                                                                    300000,
                                                                    900000]
                    }

initial_param_grid = params_to_evolve
other_params_to_sweep = params_for_setup_only

In [4]:
new_param_grid = deepcopy(initial_param_grid)
new_best_name = ""
new_best_fitness = -np.inf
times_repeat_best_name = 0
times_repeat_best_fitness = 0

old_param_grid = deepcopy(new_param_grid)
old_best_name = new_best_name
old_best_fitness = new_best_fitness

individual_dict = build_individual_dict_from_param_grid(old_param_grid)
        
        

data = kpis
data = give_data_individual_names(data)

new_best_name, new_best_val = find_best_individual_name(
            data, fitness_func=fitness_func, name_biased_towards=old_best_name
        )

assert not (
            new_best_name is None
        ), "The best individual should be somewhere in this dictionary."
print(new_best_name)

new_best_individual = individual_dict[new_best_name]

if new_best_name == old_best_name:
    times_repeat_best_name = times_repeat_best_name + 1
else:
    times_repeat_best_name = 0

if isclose(new_best_fitness, old_best_fitness):
    times_repeat_best_fitness = times_repeat_best_fitness + 1
else:
    times_repeat_best_fitness = 0
    
new_param_grid = update_param_grid(
                old_param_grid=old_param_grid, best_individual=new_best_individual
            )

Building the individual dict.
Individual dict is {'stbc:25_gfpr:20_afpr:20_gms:100000000000.0_ams:100000000000.0': {'session_token_bucket_coefficient': 25, 'gateway_fee_per_relay': 20, 'application_fee_per_relay': 20, 'gateway_minimum_stake': 100000000000.0, 'application_minimum_stake': 100000000000.0}, 'stbc:25_gfpr:20_afpr:20_gms:100000000000.0_ams:200000000000.0': {'session_token_bucket_coefficient': 25, 'gateway_fee_per_relay': 20, 'application_fee_per_relay': 20, 'gateway_minimum_stake': 100000000000.0, 'application_minimum_stake': 200000000000.0}, 'stbc:25_gfpr:20_afpr:20_gms:200000000000.0_ams:100000000000.0': {'session_token_bucket_coefficient': 25, 'gateway_fee_per_relay': 20, 'application_fee_per_relay': 20, 'gateway_minimum_stake': 200000000000.0, 'application_minimum_stake': 100000000000.0}, 'stbc:25_gfpr:20_afpr:20_gms:200000000000.0_ams:200000000000.0': {'session_token_bucket_coefficient': 25, 'gateway_fee_per_relay': 20, 'application_fee_per_relay': 20, 'gateway_minimum_

In [5]:
new_param_grid

{'session_token_bucket_coefficient': [25, 212.5],
 'gateway_fee_per_relay': [25.0, 30],
 'application_fee_per_relay': [20, 25.0],
 'gateway_minimum_stake': [150000000000.0, 200000000000.0],
 'application_minimum_stake': [100000000000.0, 150000000000.0]}

In [6]:
old_param_grid

{'session_token_bucket_coefficient': [25, 400],
 'gateway_fee_per_relay': [20, 30],
 'application_fee_per_relay': [20, 30],
 'gateway_minimum_stake': [100000000000.0, 200000000000.0],
 'application_minimum_stake': [100000000000.0, 200000000000.0]}